In [ ]:
!pip install tweepy praw requests beautifulsoup4 google-api-python-client transformers geopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 3.2 MB/s eta 0:00:00


In [ ]:
reddit_client_id = "S2w8w5MVgGfECmkkQjaiIA"
reddit_client_secret = "oLNAsFmEHONbv8V0mf29hu6riJsUDQ"
reddit_user_agent = "ThreatSentryAI/1.0 (by /u/your_reddit_username)"

In [ ]:
import praw

reddit = praw.Reddit(client_id=reddit_client_id, client_secret=reddit_client_secret, user_agent=reddit_user_agent)
reddit_posts = []
for submission in reddit.subreddit("worldnews+geopolitics").search("India Pakistan conflict", limit=50):
    reddit_posts.append(submission.title + " " + submission.selftext)
print("Reddit Posts:", reddit_posts[:5])


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Reddit Posts: ['Israel sends precision bombs to India as conflict with Pakistan intensifies ', 'Chinese Perspectives on Conflict with India Hello! posting on an alt account for privacy reasons but I am an Chinese American college student majoring in international studies at JHU SAIS. I have translated and summarized one of the most popular posts on Zhihu (China\'s Quora) about Chinese- Indian relations. I feel this is useful information to share as it summarizes and explains the CCP\'s current view of India (accurate based on my personal links to CCP) and explains why China behaves so antagonistically. This post is **LONG** but is still much shorter than the original. The original has 9 chapters, chapters 5 - 9 focusing on Chinese military strategy in a Sino- Indian war. In the r/india post I have focused on the politics and reasons for conflict described in chapters 1-4. **For** r/geopolitics **I have added a shortened version of chapter 5 describing the importance of Taiwan in Chines

In [ ]:
import requests
from bs4 import BeautifulSoup

quora_posts = []
url = "https://www.quora.com/search?q=India+Pakistan+conflict"
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")
for post in soup.find_all("div", class_="puppeteer_test_question_title", limit=10):
    quora_posts.append(post.text)
print("Quora Posts:", quora_posts[:5])

Quora Posts: []


In [ ]:
!pip install transformers

In [ ]:
# Assuming your Reddit and Quora data are in lists named reddit_posts and quora_posts
all_posts = reddit_posts + quora_posts
print("Total Posts:", len(all_posts))
print("Sample Posts:", all_posts[:5])

Total Posts: 50
Sample Posts: ['Israel sends precision bombs to India as conflict with Pakistan intensifies ', 'Chinese Perspectives on Conflict with India Hello! posting on an alt account for privacy reasons but I am an Chinese American college student majoring in international studies at JHU SAIS. I have translated and summarized one of the most popular posts on Zhihu (China\'s Quora) about Chinese- Indian relations. I feel this is useful information to share as it summarizes and explains the CCP\'s current view of India (accurate based on my personal links to CCP) and explains why China behaves so antagonistically. This post is **LONG** but is still much shorter than the original. The original has 9 chapters, chapters 5 - 9 focusing on Chinese military strategy in a Sino- Indian war. In the r/india post I have focused on the politics and reasons for conflict described in chapters 1-4. **For** r/geopolitics **I have added a shortened version of chapter 5 describing the importance of 

In [ ]:
from transformers import pipeline

# Initialize sentiment analysis model
classifier = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

# Analyze posts
threats = []
for post in all_posts:
    try:
        result = classifier(post[:512])[0]  # Truncate to 512 chars (model limit)
        if result["label"] == "NEGATIVE" and result["score"] > 0.9:  # High-confidence negative
            threats.append({"text": post, "confidence": result["score"]})
    except Exception as e:
        print(f"Error analyzing post: {e}")
        continue

# Print results
print("Potential Threats:", len(threats))
for threat in threats[:5]:
    print(f"Threat: {threat['text'][:100]}... (Confidence: {threat['confidence']:.2f})")

Device set to use cpu


Potential Threats: 24
Threat: Pakistani PM Imran Khan appeals for talks with India to avoid war. Kashmir preparing for conflict af... (Confidence: 0.93)
Threat: Indian Army destroys 4 Pakistani posts in massive assault in Keran, Jammu and Kashmir. ... (Confidence: 1.00)
Threat: Will Pakistan go to war (like the 1999 conflict) with India over Kashmir ? Pakistan PM wrote: “If th... (Confidence: 0.99)
Threat: Iran ready to halt 20% uranium enrichment, West must reciprocate "Iran has confirmed it is prepared ... (Confidence: 0.98)
Threat: Stand-off in Kashmir: ‘Our last hope is that a war will sort this once and for all’ As tensions grow... (Confidence: 0.99)


In [ ]:
# Initialize NER model
ner = pipeline("ner", model="dslim/bert-base-NER")

# Extract entities from threats
entities = []
for threat in threats:
    try:
        ner_results = ner(threat["text"][:512])  # Truncate to 512 chars
        for entity in ner_results:
            if entity["entity"].startswith("B-LOC"):  # Location entities
                entities.append({"text": threat["text"], "location": entity["word"], "confidence": threat["confidence"]})
    except Exception as e:
        print(f"Error extracting entities: {e}")
        continue

# Print results
print("Extracted Locations:", len(entities))
for entity in entities[:5]:
    print(f"Text: {entity['text'][:100]}... | Location: {entity['location']} | Confidence: {entity['confidence']:.2f}")

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


Extracted Locations: 87
Text: Pakistani PM Imran Khan appeals for talks with India to avoid war. Kashmir preparing for conflict af... | Location: India | Confidence: 0.93
Text: Pakistani PM Imran Khan appeals for talks with India to avoid war. Kashmir preparing for conflict af... | Location: Kashmir | Confidence: 0.93
Text: Indian Army destroys 4 Pakistani posts in massive assault in Keran, Jammu and Kashmir. ... | Location: Ke | Confidence: 1.00
Text: Indian Army destroys 4 Pakistani posts in massive assault in Keran, Jammu and Kashmir. ... | Location: ##ran | Confidence: 1.00
Text: Indian Army destroys 4 Pakistani posts in massive assault in Keran, Jammu and Kashmir. ... | Location: Jammu | Confidence: 1.00


In [ ]:
# Install transformers (run once)
!pip install transformers

# Combine Reddit and Quora data
all_posts = reddit_posts + quora_posts  # Replace with your variable names if different
print("Total Posts:", len(all_posts))
print("Sample Posts:", all_posts[:5])

# Threat detection with toxic-bert
from transformers import pipeline
classifier = pipeline("text-classification", model="unitary/toxic-bert")
threats = []
for post in all_posts:
    try:
        result = classifier(post[:512])[0]
        if result["label"] == "toxic" and result["score"] >= 0.6:
            threats.append({"text": post, "confidence": result["score"]})
    except Exception as e:
        print(f"Error analyzing post: {e}")
        continue
print("Potential Threats:", len(threats))
for threat in threats[:5]:
    print(f"Threat: {threat['text'][:100]}... (Confidence: {threat['confidence']:.2f})")

# Entity extraction (locations)
ner = pipeline("ner", model="dslim/bert-base-NER")
entities = []
for threat in threats:
    try:
        ner_results = ner(threat["text"][:512])
        for entity in ner_results:
            if entity["entity"].startswith("B-LOC"):
                entities.append({
                    "text": threat["text"],
                    "location": entity["word"],
                    "confidence": threat["confidence"]
                })
    except Exception as e:
        print(f"Error extracting entities: {e}")
        continue
print("Extracted Locations:", len(entities))
for entity in entities[:5]:
    print(f"Text: {entity['text'][:100]}... | Location: {entity['location']} | Confidence: {entity['confidence']:.2f}")

Total Posts: 50
Sample Posts: ['Israel sends precision bombs to India as conflict with Pakistan intensifies ', 'Chinese Perspectives on Conflict with India Hello! posting on an alt account for privacy reasons but I am an Chinese American college student majoring in international studies at JHU SAIS. I have translated and summarized one of the most popular posts on Zhihu (China\'s Quora) about Chinese- Indian relations. I feel this is useful information to share as it summarizes and explains the CCP\'s current view of India (accurate based on my personal links to CCP) and explains why China behaves so antagonistically. This post is **LONG** but is still much shorter than the original. The original has 9 chapters, chapters 5 - 9 focusing on Chinese military strategy in a Sino- Indian war. In the r/india post I have focused on the politics and reasons for conflict described in chapters 1-4. **For** r/geopolitics **I have added a shortened version of chapter 5 describing the importance of 

config.json:   0%|          | 0.00/811 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


Potential Threats: 0


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


Extracted Locations: 0


In [ ]:
# Replace with your actual variable names if different
all_posts = reddit_posts + quora_posts
print("Total Posts:", len(all_posts))
print("Sample Posts:", all_posts[:5])

Total Posts: 50
Sample Posts: ['Israel sends precision bombs to India as conflict with Pakistan intensifies ', 'Chinese Perspectives on Conflict with India Hello! posting on an alt account for privacy reasons but I am an Chinese American college student majoring in international studies at JHU SAIS. I have translated and summarized one of the most popular posts on Zhihu (China\'s Quora) about Chinese- Indian relations. I feel this is useful information to share as it summarizes and explains the CCP\'s current view of India (accurate based on my personal links to CCP) and explains why China behaves so antagonistically. This post is **LONG** but is still much shorter than the original. The original has 9 chapters, chapters 5 - 9 focusing on Chinese military strategy in a Sino- Indian war. In the r/india post I have focused on the politics and reasons for conflict described in chapters 1-4. **For** r/geopolitics **I have added a shortened version of chapter 5 describing the importance of 

In [ ]:
from transformers import pipeline

# Initialize toxic comment classifier
classifier = pipeline("text-classification", model="unitary/toxic-bert")

# Analyze posts for threats
threats = []
for post in all_posts:
    try:
        result = classifier(post[:512])[0]  # Truncate to 512 chars (model limit)
        if result["label"] == "toxic" and result["score"] >= 0.6:  # Threshold 0.6
            threats.append({"text": post, "confidence": result["score"]})
    except Exception as e:
        print(f"Error analyzing post: {e}")
        continue

# Print results
print("Potential Threats:", len(threats))
for threat in threats[:5]:
    print(f"Threat: {threat['text'][:100]}... (Confidence: {threat['confidence']:.2f})")

Device set to use cpu


Potential Threats: 0
